***Fake News Prediction in ML using Logistic Regression model***

importing the dependencies

In [1]:
import pandas as pd
import numpy as np 
import re
from nltk.corpus import stopwords
from nltk.stem.porter import  PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import  train_test_split
from sklearn.linear_model import  LogisticRegression
from sklearn.metrics import  accuracy_score



dowload the stopwords by using natural language toolkit(nltk)

In [2]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to C:\Users\Madina
[nltk_data]     Computers\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

check to find which are the stopwords so that they will remove from the any text data

In [76]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

loading the dataset

In [4]:
news_dataset = pd.read_csv('fake_news_dataset.csv')
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


news_dataset.csv: A full news dataset with the following attributes:

id: unique id for a news article

title: the title of a news article

author: author of the news article

text: the text of the article; could be incomplete

label: a label that marks the article as potentially unreliable

1 = unreliable(fake)

0 = reliable(real)

In [5]:
news_dataset.info

<bound method DataFrame.info of           id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2      

In [6]:
news_dataset.describe()

,id,label
count,20800.000000,20800.000000
mean,10399.500000,0.500625
std,6004.587135,0.500012
min,0.000000,0.000000
25%,5199.750000,0.000000
50%,10399.500000,1.000000
75%,15599.250000,1.000000
max,20799.000000,1.000000


check the number of missing values in our dataset either present or not if present then how many?

In [7]:
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

so there is null values in our dataset but not too much that can affect our precess or prediction so that we replace it with 'Nan' values

In [8]:
news_dataset = news_dataset.fillna('')

In [9]:
news_dataset.isnull().sum()


id        0
title     0
author    0
text      0
label     0
dtype: int64

for all 3 features ,but it will slow down our processing

In [11]:
# news_dataset['content'] = news_dataset['author']+ ' ' +news_dataset['title']+' ' +news_dataset['text']

for 2 features, cancatenate them !

In [14]:
news_dataset['content'] = news_dataset['author']+ '     ' +news_dataset['title']

In [15]:
news_dataset['content']

0        Darrell Lucus     House Dem Aide: We Didn’t Ev...
1        Daniel J. Flynn     FLYNN: Hillary Clinton, Bi...
2        Consortiumnews.com     Why the Truth Might Get...
3        Jessica Purkiss     15 Civilians Killed In Sin...
4        Howard Portnoy     Iranian woman jailed for fi...
                               ...                        
20795    Jerome Hudson     Rapper T.I.: Trump a ’Poster...
20796    Benjamin Hoffman     N.F.L. Playoffs: Schedule...
20797    Michael J. de la Merced and Rachel Abrams     ...
20798    Alex Ansary     NATO, Russia To Hold Parallel ...
20799          David Swanson     What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object

separating the features(x) and labels(y)

In [17]:
x = news_dataset.drop(columns='label' , axis=1)
y = news_dataset['label']

In [20]:
x[:5]

,id,title,author,text,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus House Dem Aide: We Didn’t Ev...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"Daniel J. Flynn FLYNN: Hillary Clinton, Bi..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Consortiumnews.com Why the Truth Might Get...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,Jessica Purkiss 15 Civilians Killed In Sin...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Howard Portnoy Iranian woman jailed for fi...


In [21]:
y[:5]

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

the module that remove unecessory things excluding specific one

In [22]:
port_stem = PorterStemmer()

## Stemming

Stemming is the process of producing morphological variants of a root/base word. Stemming programs are commonly referred to as stemming algorithms or stemmers. A stemming algorithm reduces the words “chocolates”, “chocolatey”, and “choco” to the root word, “chocolate” and “retrieval”, “retrieved”, “retrieves” reduce to the stem “retrieve”.

Some more example of stemming for root word "like" include:

-> "likes"
-> "liked"
-> "likely"
-> "liking"

function for stemming

In [29]:
def stemming(content):
    #substituting by the use of regular expression's function
    stemmed_data = re.sub('[^a-zA-Z]',' ',content)
    #converting all text into lower case alphabets
    stemmed_data = stemmed_data.lower()
    #split() method splits a string into a list
    stemmed_data = stemmed_data.split()
    #removing stopwords
    stemmed_data = [port_stem.stem(word) for word in stemmed_data if not word in stopwords.words('english')]
    #collecting words other than stopwords 
    stemmed_data = ' '.join(stemmed_data)
    return stemmed_data

apply this stemming function on content column which we are created by the cancatination of 'author' and 'title' columns

In [30]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [31]:
news_dataset['content']

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object

so in the above output there are no stopwords , uppercase words and signs 

Now we are separting the feature(x) and lable(y) values as we did it before on the basis of author and title but now the x will based on 'content' column.

In [33]:
x = news_dataset['content'].values
y = news_dataset['label'].values

In [37]:
print(x,x.shape)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv'] (20800,)


In [38]:
print(y,y.shape)

[1 0 1 ... 0 1 1] (20800,)


## What is TfidfVectorizer()

The term tf–idf stands for term frequency–inverse document frequency. It is a mathematical statistic that is planned to reflect how significant a word is to a record in a collection or corpus. The tf–idf esteem builds proportionally to the number of times a word shows up in the document. It is offset by the quantity of documents in the corpus that contain the word, which helps to adjust for the fact that a few words show up more often when all is said and done. TF–IDF is one of the most well-known term-weighting plans today. An overview led in 2015 demonstrated that 83% of text-based recommender frameworks in advanced libraries use tf–idf. It would be difficult to understand tf–idf together. So, let's understand each separately-






Term Frequency (tf)- It gives us the recurrence of the word in each report in the corpus. It is the proportion of the number of times the word shows up in a report contrasted with the all-out the number of words in that record. It increments as the quantity of events of that word inside the record increments.

Inverse Data Frequency (idf)- It is used to figure out the heaviness of uncommon words over all reports in the corpus. The words that occur seldom in the corpus have a high IDF score.

Joining these two, we think of the TF-IDF score (w) for a word in a record in the corpus.






TD-IDF Example
Let's take an example(A,B respectivily) to get more clear understanding.

The cycle is ridden on the track.

The bus is driven on the road.

Let's assume the above two sentences are a separate document. Here, we have calculated the TF-IDF for the above two documents, which represent our corpus.

Word	TF		IDF		TF	*	IDF

 	    A B	 		A		B

The	1/7	 1/7	log(2/2) = 0	       	0	0

cycle 1/7	0	log(2/1) = 0.3		0.043	0

bus	0	1/7	log(2/1) = 0.3		0	0.043

is	1/7	 1/7	log(2/2) = 0		0	0

ridden	1/7	0	log(2/1) = 0.3		0.043	0

driven	0	1/7	log(2/1) = 0.3		0	0.043

on	1/7	1/7	log(2/2) = 0		0	0

the	1/7	1/7	log(2/2) = 0		0	0

track	1/7	0	log(2/1) = 0.3		0.043	0

road	0	1/7	log(2/1) = 0.3		0	0.043


In the above table, we can see that TF-IDF of common words was zero, which shows they are not significant. On the other hand, the TF-IDF of "cycle" , "bus", "ridden", "driven", "track", and "road" are non-zero. These words have more significance.

####  Scikit-learn TfidfVectorizer

Scikit-learn is a free software machine learning library for the Python programming language. It supports Python numerical and scientific libraries, in which TfidfVectorizer is one of them. It converts a collection of raw documents to a matrix of TF-IDF features. As tf–idf is very often used for text features, the class TfidfVectorizer combines all the options of CountVectorizer and TfidfTransformer into a single model. The TfidfVectorizer uses an in-memory vocabulary (a python dict) to map the most frequent words to feature indices and hence compute a word occurrence frequency (sparse) matrix.



converting the textual data into numeric data

In [39]:
 vectorizer = TfidfVectorizer()
 vectorizer.fit(x)

 x = vectorizer.transform(x)

In [47]:
print(x)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

train_test_splitting

In [48]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y, random_state = 2 )

In [49]:
print(x_train, x_train.shape)

  (0, 16996)	0.09995656818816077
  (0, 16928)	0.3360072514082535
  (0, 15295)	0.09807665903342763
  (0, 13914)	0.3334797245354899
  (0, 13014)	0.2680313811122545
  (0, 12501)	0.3929876463935473
  (0, 11936)	0.24142639024498436
  (0, 10306)	0.09662001419895176
  (0, 10219)	0.3019527708144002
  (0, 3155)	0.3400831511004003
  (0, 2794)	0.3776836172783757
  (0, 336)	0.3360072514082535
  (1, 16996)	0.07263181421455335
  (1, 15424)	0.22579404836928033
  (1, 15417)	0.26613170238131584
  (1, 15295)	0.07126580880898774
  (1, 13453)	0.3387500815971264
  (1, 11421)	0.3084666283145136
  (1, 10306)	0.07020736153621741
  (1, 10061)	0.24924889065491332
  (1, 9856)	0.3387500815971264
  (1, 5313)	0.2768869285533855
  (1, 2761)	0.3387500815971264
  (1, 2207)	0.3387500815971264
  (1, 956)	0.2855580802628186
  :	:
  (16637, 10008)	0.32617532466383187
  (16637, 9588)	0.184164927958451
  (16637, 8153)	0.3214731053599403
  (16637, 5009)	0.3538476463585765
  (16637, 1868)	0.40919228974806576
  (16637, 364)	0.

In [50]:
print(x_test, x_test.shape)


  (0, 12801)	0.2910746804557067
  (0, 9818)	0.30786004182651133
  (0, 7668)	0.22945314906455008
  (0, 6816)	0.16094563145945953
  (0, 6289)	0.288254092437116
  (0, 5941)	0.288254092437116
  (0, 5233)	0.21316265672448448
  (0, 4346)	0.3250084367199054
  (0, 3395)	0.3301936745912874
  (0, 2959)	0.24534646237198773
  (0, 1667)	0.30373060380734146
  (0, 908)	0.213510750423647
  (0, 239)	0.34297808354766485
  (1, 16996)	0.09117761343372983
  (1, 15295)	0.08946281236254729
  (1, 14046)	0.42524648908354634
  (1, 13190)	0.36773046084789346
  (1, 12741)	0.24868518461414146
  (1, 12279)	0.3796661151115819
  (1, 12041)	0.37327055071909065
  (1, 10306)	0.08813410128297053
  (1, 8813)	0.42524648908354634
  (1, 4008)	0.23098933893199997
  (1, 3339)	0.2834482751186189
  (2, 16868)	0.344315415802567
  :	:
  (4158, 16782)	0.2796201559111399
  (4158, 13954)	0.2713157796360236
  (4158, 13918)	0.29432702492466434
  (4158, 13775)	0.21479309786895925
  (4158, 13600)	0.2496550139728973
  (4158, 11068)	0.2943

In [51]:
print(y_train, y_train.shape)


[0 0 0 ... 0 0 1] (16640,)


In [52]:
print(y_test, y_test.shape)


[1 0 1 ... 1 1 0] (4160,)


training the model

In [53]:
model = LogisticRegression()


### For training data


In [55]:
#for training data
model.fit(x_train,y_train)

LogisticRegression()

prediction for training data

In [56]:
prediction_of_training_data = model.predict(x_train)
prediction_of_training_data

array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

### Evaluation(checking the accuracy score of our model on the training data & test data)

##### Accuracy score for training data

In [59]:
accuracy_of_training_prediction = accuracy_score(prediction_of_training_data,y_train)
print("The accuracy score of traing data is :",accuracy_of_training_prediction)

The accuracy score of traing data is : 0.9865985576923076


In [62]:
#for test data
model.fit(x_test,y_test)

LogisticRegression()

prediction for test data

In [63]:
#for test data
prediction_of_test_data = model.predict(x_test)
prediction_of_test_data

array([1, 0, 1, ..., 1, 1, 0], dtype=int64)

Accuracy for test data

In [77]:
accuracy_of_test_prediction = accuracy_score(prediction_of_test_data,y_test)
print("The accuracy score of test data is :",accuracy_of_test_prediction)

The accuracy score of test data is : 0.9795673076923077


### System Implementation

In [73]:
#so first take the very first news from the test data as we can see that this first news is fake = 1

x_new = x_test[0]
x_new_prediction = model.predict(x_new)
print(x_new_prediction)

if x_new_prediction == 1:
    print("This news is fake !")
else:
    print("This news is orignal or not fake ! ")

[1]
This news is fake !


In [69]:
#so take the 2nd news from the test data as we can see in test data that this 2nd news is orignal = 0

x_new = x_test[1]
x_new_prediction = model.predict(x_new)
print(x_new_prediction)


if x_new_prediction == 1:
    print("This news is fake !")
else:
    print("This news is orignal or not fake !")

[0]
This news is orignal or not fake !


***Hence Completed !***

___________________ Best Wishes & Hope for the best ___________________